In [2]:
# There's a dependency in Bloqade that (currently)
# spits out some warnings you don't need to worry about.
import warnings
warnings.filterwarnings("ignore")

from bloqade.qasm2.rewrite.native_gates import RydbergGateSetRewriteRule
from kirin.rewrite import Walk
from bloqade import qasm2
from bloqade.qasm2.parse.lowering import QASM2
from bloqade.qasm2.passes import QASM2Py
from bloqade.qasm2.emit import QASM2 # the QASM2 target
from bloqade.qasm2.parse import pprint # the QASM2 pretty printer

from kirin import ir
from qiskit import QuantumCircuit

from bloqade.qasm2.rewrite.native_gates import RydbergGateSetRewriteRule
from kirin import ir
from kirin.rewrite import Walk
from bloqade.qasm2.passes import UOpToParallel, QASM2Fold

from bloqade.qasm2.emit import QASM2 # the QASM2 target
from bloqade.qasm2.parse import pprint # the QASM2 pretty printer

target = QASM2(allow_parallel=True)

import numpy as np


@ir.dialect_group(qasm2.extended)
def extended_opt(self):
    native_rewrite = Walk(RydbergGateSetRewriteRule(self)) # use Kirin's functionality to walk code line by line while applying neutral-atom gate decomposition as defined in Bloqade
    parallelize_pass = UOpToParallel(self) # review the code and apply parallelization using a heuristic
    agg_fold = QASM2Fold(self) # supports parallelization by unfolding loops to search for parallelization opportunities

    # here we define our new compiler pass
    def run_pass(
        kernel: ir.Method,
        *,
        fold: bool = True,
        typeinfer: bool = True,
        parallelize: bool = False,
    ):
        assert qasm2.extended.run_pass is not None
        qasm2.extended.run_pass(kernel, fold=fold, typeinfer=typeinfer) # apply the original run_pass to the lowered kernel
        native_rewrite.rewrite(kernel.code) # decompose all gates in the circuit to neutral atom gate set

        # here goes our parallelization optimizer; the order of the commands here matters!
        if parallelize:
            agg_fold.fixpoint(kernel)
            parallelize_pass(kernel)

    return run_pass


# helper to go from Method → Qiskit
def method_to_qiskit(method: ir.Method, *,parallelize: bool = True) -> QuantumCircuit:
    # run extended pass in case forgot
    extended_opt.run_pass(method, parallelize=parallelize)
    # emit OpenQASM2 text
    qasm = QASM2().emit_str(method)
    # parse into a Qiskit circuit
    return QuantumCircuit.from_qasm_str(qasm)

def method_to_qiskit_noparallel(method: ir.Method, *,parallelize: bool = True) -> QuantumCircuit:
    # emit OpenQASM2 text
    qasm = QASM2().emit_str(method)
    # parse into a Qiskit circuit
    return QuantumCircuit.from_qasm_str(qasm)

In [3]:
def split(file, lst):
    if len(lst) <= 1:
        return 0
    mid =(len(lst)-1)//2 
    file.write(f"cx q[{lst[-1]}], q[{lst[mid]}];"+"\n")
    split(file, lst[:mid+1])
    split(file, lst[mid+1:])
    return 0

def desplit(file, lst):
    if len(lst) <= 1:
        return 0
    for i in range(len(lst)-1,0,-2):
        file.write(f"cx q[{lst[i]}], q[{lst[i-1]}];"+"\n")
        lst.pop(i-1)
    desplit(file, lst)
    return 0
     

def write_circuit(n, f, beta = np.pi/4, alpha = np.pi/4):
    f.write("OPENQASM 2.0;\n")
    f.write("include \"qelib1.inc\";\n")
    f.write("qreg q[{}];\n".format(n))
    for i in range(n):
        # hadamards
        f.write(f"h q[{i}];"+"\n")
    # first round of big blocks
    for m in range(n-1,0,-1):
        # cx ladder 
        for k in range(m):
            # c rz(theta)
            #f.write(f"u3(0, {beta/2-np.pi/2}, {np.pi/2}) q[{m}];"+"\n")
            f.write(f"cx q[{m}], q[{k}];"+"\n")
            f.write(f"u3(0, {-beta/2-np.pi/2}, {np.pi/2}) q[{k}];"+"\n")
            f.write(f"cx q[{m}], q[{k}];"+"\n")
            f.write(f"u3(0, {beta/2-np.pi/2}, {np.pi/2}) q[{k}];"+"\n")
    # rx phases
    for i in range(n):
        f.write(f"u3({alpha}, {-np.pi/2}, {np.pi/2}) q[{i}];"+"\n")
        
    # second round
    for m in range(n-1,0,-1):
        # cx ladder 
        for k in range(m):
            # c rz(theta)
            #f.write(f"u3(0, {beta/2-np.pi/2}, {np.pi/2}) q[{m}];"+"\n")
            f.write(f"cx q[{m}], q[{k}];"+"\n")
            f.write(f"u3(0, {-beta/2-np.pi/2}, {np.pi/2}) q[{k}];"+"\n")
            f.write(f"cx q[{m}], q[{k}];"+"\n")
            f.write(f"u3(0, {beta/2-np.pi/2}, {np.pi/2}) q[{k}];"+"\n")
    # rx phases
    for i in range(n):
        f.write(f"u3({alpha}, {-np.pi/2}, {np.pi/2}) q[{i}];"+"\n")

In [4]:
with open("qaoa_base1.qasm", "w+") as f:
    write_circuit(4, f)